In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Reading the Weather and PM2.5 Data

- using min-max normalization

In [8]:
from data_preprocessing import load_X_y
import pandas as pd

weather_df, pollutant_df = load_X_y()

In [9]:
weather_df_normalized = (weather_df-weather_df.min())/(weather_df.max()-weather_df.min())
weather_df_normalized.describe()

,max_dew_point_v,max_relative_humidity_v,max_temperature_v,max_wind_speed_v,min_dew_point_v,min_relative_humidity_v,min_temperature_v,min_wind_speed_v,precipitation_v,rain_v,snow_v,snow_on_ground_v
count,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000,7304.000000
mean,0.514953,0.571856,0.524171,0.571205,0.518990,0.486371,0.519177,0.510449,0.491174,0.509921,0.247726,0.177910
std,0.335249,0.218783,0.335174,0.255798,0.326785,0.261585,0.314073,0.271372,0.181621,0.244698,0.315199,0.278788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.179659,0.412934,0.182859,0.357715,0.201610,0.258465,0.210211,0.271762,0.344340,0.237410,0.000000,0.000000
50%,0.495475,0.597683,0.536310,0.632265,0.500370,0.435290,0.517430,0.509554,0.504717,0.593525,0.016667,0.000000
75%,0.863534,0.743629,0.859094,0.795591,0.851591,0.731753,0.828169,0.762208,0.617925,0.701439,0.583333,0.360465
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
pollutant_df

,P2.5
0,9.208333
1,8.750000
2,8.750000
3,11.000000
4,12.833333
...,...
7299,5.958333
7300,8.500000
7301,5.041667
7302,2.916667


In [15]:
pollutant_tensor = torch.tensor(pollutant_df['P2.5'].values)
weather_tensor = torch.tensor(weather_df_normalized[:].values)

print(pollutant_tensor.shape)
print(weather_tensor.shape)

torch.Size([7304])
torch.Size([7304, 12])


In [17]:
type(pollutant_tensor)

torch.Tensor

# Dataset

- We'll train using the first 5000+14 days to train, 1000+14 days to validate, 1000+14 days to test
- Ensure sequencing is maintained
- Test data will be the latest data as the model is intended to use historical data to predict future data

In [23]:
from torch.utils.data import Dataset

In [44]:
# creating a custom dataset in a sliding window manner
class WeatherPollutantDataset(Dataset):
    def __init__(self, weather: torch.Tensor, pollutant: torch.Tensor, window:int):
        self.weather = weather
        self.pollutant = pollutant
        # assumes the data starts on the same day
        # assumes their length is the same
        assert len(self.weather) == len(self.pollutant)
        self.window = window

    def __getitem__(self, index: int):
        """
        Using weather input on day i,i+1,...,i+window-1 to predict pollutant output on i+window
        """
        weather_input = self.weather[index:index+self.window].permute(1,0)
        pollutant_output = self.pollutant[index+self.window]
        return weather_input, pollutant_output

    def __len__(self):
        return len(self.weather) - self.window

In [45]:
train_set = WeatherPollutantDataset(weather=weather_tensor[:5014], pollutant=pollutant_tensor[:5014], window=14)
val_set   = WeatherPollutantDataset(weather=weather_tensor[5014:6028], pollutant=pollutant_tensor[5014:6028], window=14)
test_set  = WeatherPollutantDataset(weather=weather_tensor[6028:7042], pollutant=pollutant_tensor[6028:7042], window=14)

In [37]:
print("Number of training data: %d" % len(train_set))
print("Number of validation data: %d" % len(val_set))
print("Number of testing data: %d" % len(test_set))

Number of training data: 5000
Number of validation data: 1000
Number of testing data: 1000


In [38]:
from torch.utils.data import DataLoader

In [46]:
test_loader = DataLoader(test_set, batch_size=64, shuffle=True)
next(iter(test_loader))[0].shape

torch.Size([64, 12, 14])

# Model

In [ ]:
class CNNLSTM(nn.Module):
    def __init__(self):
        super(self).__init__()
        self.name = "CNNLSTM"
        self.pool = torch.nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv1 = torch.nn.conv1d(in_channels=12, out_channels=16, kernel_size=3, stride=1, padding=0)
        self.conv2 = torch.nn.conv1d(in_channels=16, out_channels=25, kernel_size=3, stride=1, padding=0)
        self.conv3 = torch.nn.conv1d(in_channels=25, out_channels=36, kernel_size=3, stride=1, padding=0)

        

    def forward(self, x):
        pass